In [1]:
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/ltnga/LawVN-Instructction-Gen/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Configuration constants
WEAVIATE_URL="https://kh0e0endr9olrknql51vuw.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY="4eLFZtLBJdh8wOc0gHC85SzxmIuTYEzcIpup"
DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 2000 
CHUNK_OVERLAP = 0 

In [13]:
# Load documents
with open("/home/ltnga/LawVN-Instructction-Gen/src/data/data.json") as f:
    data = json.load(f)
documents = [Document(text=doc) for doc in data]

In [14]:
# Create chunker
chunker = SentenceSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separator=" ",
    include_prev_next_rel=False,  # Keep relationships between chunks
)

In [17]:
# Process nodes
nodes = chunker.get_nodes_from_documents(documents, show_progress=True)
# for node in nodes:
#     node.metadata["original_text"] = node.text  # Store original text
#     node.text = ViTokenizer.tokenize(node.text.lower())  # Tokenize Vietnamese text
#     node.excluded_embed_metadata_keys.append("original_text")
#     node.excluded_llm_metadata_keys.append("original_text")

Parsing nodes: 100%|██████████| 51/51 [00:00<00:00, 468.85it/s]


In [18]:
nodes[38].text

'Ngoài việc bị áp dụng hình thức xử phạt, người điều khiển phương tiện thực hiện hành vi quy định tại điểm c khoản 5, điểm d khoản 7, điểm c khoản 8 Điều này bị trừ điểm giấy phép lái xe được cấp mới nhất trong hệ thống thông tin quản lý giấy phép lái xe 02 điểm.'

In [19]:
nodes[37].text

'Phạt tiền từ 2.000.000 đồng đến 4.000.000 đồng đối với người điều khiển xe mô tô hai bánh có dung tích xi-lanh đến 125 cm3 hoặc có công suất động cơ điện đến 11 kW và các loại xe tương tự xe mô tô thực hiện một trong các hành vi vi phạm sau đây:\na) Không có giấy phép lái xe hoặc sử dụng giấy phép lái xe đã bị trừ hết điểm hoặc sử dụng giấy phép lái xe không do cơ quan có thẩm quyền cấp, giấy phép lái xe bị tẩy xóa, giấy phép lái xe không còn hiệu lực, giấy phép lái xe không phù hợp với loại xe đang điều khiển;\nb) Có giấy phép lái xe quốc tế do các nước tham gia Công ước của Liên hợp quốc về Giao thông đường bộ năm 1968 cấp (trừ giấy phép lái xe quốc tế do Việt Nam cấp) nhưng không mang theo giấy phép lái xe quốc gia phù hợp với loại xe được phép điều khiển;\nc) Sử dụng giấy phép lái xe không hợp lệ (giấy phép lái xe có số phôi ghi ở mặt sau không trùng với số phôi được cấp mới nhất trong hệ thống thông tin quản lý giấy phép lái xe).\n6. Phạt tiền từ 4.000.000 đồng đến 6.000.000 đồng

In [12]:
# Setup vector store
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [13]:
# Create embedding model
embed_model = HuggingFaceEmbedding(
    model_name=MODEL_NAME,
    max_length=8192,
    device=DEVICE,
    trust_remote_code=True
)

A new version of the following files was downloaded from https://huggingface.co/dangvantuan/Vietnamese_impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/dangvantuan/Vietnamese_impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [14]:
# Build index
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    insert_batch_size=32768,  # Optimized batch size
    show_progress=True
)

Generating embeddings: 100%|██████████| 98/98 [00:03<00:00, 32.52it/s]


In [21]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=5,
    alpha=0.2  # Balance between keyword and semantic search
)

In [24]:
query = "Hành vi chở trẻ em dưới 10 tuổi và chiều cao dưới 1,35 mét trên ô tô mà không sử dụng thiết bị an toàn phù hợp sẽ bị phạt tiền"
results = retriever.retrieve(query)
for i, node in enumerate(results):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print(f"Token Embedding: {node.embeddings}\n")
    print("="*50)


Result 1:

Text: 3 . phạt tiền từ 800.000 đồng đến 1.000.000 đồng đối_với người điều_khiển xe thực_hiện một trong các hành_vi vi_phạm sau đây : a ) điều_khiển xe chạy quá tốc_độ quy_định từ 05 km / h đến dưới 10 km / h ; b ) sử_dụng còi , rú ga liên_tục ; sử_dụng còi hơi , sử_dụng đèn_chiếu xa khi gặp người đi bộ qua đường hoặc khi đi trên đoạn đường qua khu dân_cư có hệ_thống chiếu sáng đang hoạt_động hoặc khi gặp xe đi ngược chiều ( trừ trường_hợp dải_phân_cách có khả_năng chống chói ) hoặc khi chuyển_hướng xe tại nơi đường giao nhau , trừ các xe ưu_tiên đang đi làm nhiệm_vụ theo quy_định ; c ) chuyển_hướng không quan_sát hoặc không bảo_đảm khoảng cách an_toàn với xe phía sau hoặc không giảm_tốc_độ hoặc không có tín_hiệu báo hướng rẽ hoặc có tín_hiệu báo hướng rẽ nhưng không sử_dụng liên_tục trong quá_trình chuyển_hướng ( trừ trường_hợp điều_khiển xe đi theo hướng cong của đoạn đường_bộ ở nơi đường không giao nhau cùng mức ) ; d ) không tuân_thủ các quy_định về dừng xe , đỗ xe tại nơ

AttributeError: 'NodeWithScore' object has no attribute 'embeddings'